In [21]:
# extracts data from American Community Survey API

import pandas as pd
import requests
import json

In [22]:
def getQuery(year, apiKey, cats, zips):
    # indexes to reorder formatted response so that zip code is the first entry
    reorder = [len(categories)] + [i for i in range(len(categories))]

    APIquery = "https://api.census.gov/data/%s/acs/acs5?key=%s&get=%s&for=zip%%20code%%20tabulation%%20area:%s" % (year, apiKey, cats, zips)

    # call the API and collect the response
    response = requests.get(APIquery)

    # load the response into a JSON, ignoring the first element which is just field labels
    formattedResponse = json.loads(response.text)[1:]

    # flip the order of the response from [population, zipcode] -> [zipcode, population]
    formattedResponse = [[item[i] for i in reorder] for item in formattedResponse]

    # store the response in a dataframe
    queryData = pd.DataFrame(columns=cols, data=formattedResponse)

    # save that dataframe to a CSV spreadsheet
    queryData.to_excel('queryData_' + str(y) + '.xlsx', index=False)
    
    return APIquery

In [23]:
categories = [
     'B01001_001E', # Total population
     'B19013_001E', # Median income
     'B25077_001E', # Median home value
     'B25064_001E', # Median gross rent ($)
     'B15011_001E', # Total population age 25+ years with a bachelor's degree or higher
#      'B15001_001E', # Total population educational attainment
     'B03002_003E', # Not Hispanic or Latino!!White alone
     'B03002_004E', # Not Hispanic or Latino!!Black or African American alone
     'B02001_004E', # American Indian and Alaska Native Alone
     'B03002_006E', # Not Hispanic or Latino!!Asian alone
     'B03002_007E', # Not Hispanic or Latino!!Native Hawaiian and Other Pacific Islander alone
     'B03002_008E', # Not Hispanic or Latino!!Some other race alone
     'B03002_009E', # Not Hispanic or Latino!!Two or more races
     'B03002_012E', # Hispanic or Latino
]
cols = ['zipcode', 'total_pop', 'median_income', 'median_home_val', 'median_gross_rent', 'edu', 'white',
        'black', 'american_indian', 'asian', 'pacific_islander', 'other_race', 'multi_race', 'hispanic_latino']

In [25]:
# years for data
years = [2014, 2015, 2016, 2017, 2018]

# API key for American Community Survey
apiKey = "4d373026d8457a6c1f928ef3500a874f3d3e9f40"

# query string for survey categories
cats = ','.join(categories)

# zip codes from Grameen data
zips = open('zipcodes.txt', 'r').readlines()
zips = [z.replace('\n', '') for z in zips]
zips = ','.join(zips)

# list to store API queries
queries = []

# query the API for each year and save the data in a csv file
for y in years:
    queries.append(getQuery(y, apiKey, cats, zips))

In [20]:
queries[0]

'https://api.census.gov/data/2014/acs/acs5?key=4d373026d8457a6c1f928ef3500a874f3d3e9f40&get=B01001_001E,B19013_001E,B25077_001E,B25064_001E,B15011_001E,B03002_003E,B03002_004E,B02001_004E,B03002_006E,B03002_007E,B03002_008E,B03002_009E,B03002_012E&for=zip%20code%20tabulation%20area:94578,94544,28212,11232,07087,10467,07093,11368,10453,11373,95132,28215,11224,11209,11220,11219,07201,07206,95127,90006,90032,11370,90023,11590,10031,10040,10466,10458,10452,90063,02128,11421,11207,10025,10457,10456,10460,02149,10023,10472,46254,46224,02150,11435,90011,28227,07114,11418,51510,68134,28206,11378,77080,78735,90201,78610,10310,10302,10314,10032,90059,90029,10027,10037,10304,00976,90044,11432,90016,07032,07047,90047,90255,11433,11365,90222,68127,33018,33014,90040,90242,90262,90062,90007,28213,90015,90019,90037,90057,90004,46226,46222,11377,78704,78744,78617,78745,78640,94703,94621,94601,94603,94605,94577,00952,10035,10468,11367,11801,11428,11429,11423,90065,11420,11369,11372,95116,95128,10301,112